# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [1]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import duckdb
import pandas as pd
import re
import polars as pl

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

In [3]:
dataset_path = '../../data/' 
dataset_file = 'competencia_01.csv'
table_name = 'competencia_01'

In [4]:
%%sql
create or replace table {{table_name}} as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


In [5]:
%%sql
CREATE OR REPLACE TABLE {{table_name}} AS
SELECT *
FROM read_csv_auto('{{dataset_path + dataset_file}}');


,Success


In [6]:
%%sql
select * from competencia_01

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,595944011,202106,1,0,0,61,38,195.70,-4057.52,655.01,...,0,0.0,-76170.03,0.0,1135,55827.37,25,0,3542.46,None
1,596125959,202101,1,0,0,35,82,-1061.16,16147.75,1038.32,...,4,0.0,-48599.05,0.0,2371,40944.36,27,0,2991.15,CONTINUA
2,596125959,202102,1,0,0,35,83,-1848.04,12316.81,1990.21,...,-3,0.0,-52620.74,0.0,2399,32595.30,20,0,2475.03,CONTINUA
3,596125959,202103,1,0,0,35,84,-4101.60,7032.60,492.90,...,1,0.0,-26719.28,0.0,2430,25266.24,12,0,3413.43,CONTINUA
4,596125959,202104,1,0,0,35,85,-1730.32,3871.81,1326.95,...,2,0.0,-45101.21,0.0,2460,89379.25,25,0,7084.92,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,699465872,202106,1,0,0,35,91,1632.69,11831.72,1015.78,...,0,0.0,-23397.33,0.0,2122,3847.44,4,0,1876.80,None
978435,699629664,202101,1,0,0,36,40,491.96,9160.63,195.14,...,4,0.0,0.00,0.0,45,0.00,0,0,340.17,CONTINUA
978436,699629664,202102,1,0,0,36,41,826.14,9256.18,465.45,...,-3,0.0,-4961.77,0.0,73,6280.84,3,0,715.53,CONTINUA
978437,699629664,202103,1,0,0,36,42,1167.11,9582.00,926.37,...,1,0.0,-16619.02,0.0,104,0.00,0,0,551.31,CONTINUA


In [7]:
con = duckdb.connect(database=":memory:")
con.execute(f"""
    CREATE OR REPLACE TABLE competencia_01 AS
    SELECT * FROM read_csv_auto('{dataset_path + dataset_file}');
""")

# 3️⃣ Obtener nombres de columnas
columns = con.execute("PRAGMA table_info('competencia_01')").df()["name"].to_list()
columns = [c for c in columns if c not in ['numero_de_cliente', 'clase_ternaria', 'foto_mes']]

# 4️⃣ Mostrar o usar en tu flujo
print(columns)

catColumns = [
    'active_quarter', 'cliente_vip', 'internet', 'tcallcenter', 'tcuentas',
    'thomebanking', 'tmobile_app', 'Master_delinquency','Visa_delinquency',
    'Master_status', 'Visa_status'
]

numColumns = [c for c in columns if c not in catColumns]

# 5️⃣ (Opcional) cerrar conexión
#con.close()

['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_

In [8]:
quantityColumns, amountColumns, regularColumns = [], [], []

for col in numColumns:
    if re.match(r"^(c|Master_c|Visa_c|Master_F|Visa_F|Master_f|Visa_f)", col):
        quantityColumns.append(col)
    elif re.match(r"^(m|Master_m|Visa_m)", col):
        amountColumns.append(col)
    else:
        regularColumns.append(col)


#print("quantityColumns:", quantityColumns)
lagDeltaColumns = numColumns + catColumns

ratioColumns = []

for col in amountColumns:
    target = None

    if re.match(r"^m", col):
        target = re.sub(r"^m", "c", col)
    elif re.match(r"^Master_m", col):
        target = re.sub(r"^Master_m", "Master_c", col)
    elif re.match(r"^Visa_m", col):
        target = re.sub(r"^Visa_m", "Visa_c", col)

    if target and target in quantityColumns:
        ratioColumns.append([col, target])


ratioColumns.append(['mtarjeta_master_consumo', 'Master_cconsumos'])
ratioColumns.append(['mtarjeta_visa_consumo', 'Visa_cconsumos'])
ratioColumns.append(['Master_mlimitecompra', 'Master_msaldototal'])
ratioColumns.append(['Visa_mlimitecompra', 'Visa_msaldototal'])

print("ratioColumns:", ratioColumns)



ratioColumns: [['mcuenta_corriente', 'ccuenta_corriente'], ['mcaja_ahorro', 'ccaja_ahorro'], ['mprestamos_personales', 'cprestamos_personales'], ['mprestamos_prendarios', 'cprestamos_prendarios'], ['mprestamos_hipotecarios', 'cprestamos_hipotecarios'], ['minversion2', 'cinversion2'], ['mcuenta_debitos_automaticos', 'ccuenta_debitos_automaticos'], ['mpagodeservicios', 'cpagodeservicios'], ['mpagomiscuentas', 'cpagomiscuentas'], ['mcajeros_propios_descuentos', 'ccajeros_propios_descuentos'], ['mtarjeta_visa_descuentos', 'ctarjeta_visa_descuentos'], ['mtarjeta_master_descuentos', 'ctarjeta_master_descuentos'], ['mcomisiones_mantenimiento', 'ccomisiones_mantenimiento'], ['mcomisiones_otras', 'ccomisiones_otras'], ['mforex_buy', 'cforex_buy'], ['mforex_sell', 'cforex_sell'], ['mtransferencias_recibidas', 'ctransferencias_recibidas'], ['mtransferencias_emitidas', 'ctransferencias_emitidas'], ['mextraccion_autoservicio', 'cextraccion_autoservicio'], ['mcheques_depositados', 'ccheques_deposita

In [9]:
def generate_lag_columns(column: str, n_lags: int = 1) -> list[str]:
    """
    Generate an array of SQL lag expressions for a given column.
    Does NOT include SELECT or FROM — only the column expressions.

    Parameters
    ----------
    column : str
        Column name to generate lag features for.
    n_lags : int, optional
        Number of lag periods to create. Default is 1.

    Returns
    -------
    list[str]
        List of SQL expressions for lagged columns.

    Example
    -------
    >>> generate_lag_columns("account_balance", 2)
    [
        "lag(account_balance,1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS account_balance_lag_1",
        "lag(account_balance,2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS account_balance_lag_2"
    ]
    """
    return [
        f"lag({column},{i}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS {column}_lag_{i}"
        for i in range(1, n_lags + 1)
    ]


In [10]:
def generate_delta_columns(column: str, n_lags: int = 1) -> list[str]:
    """
    Generate an array of SQL expressions for delta (difference) columns
    based on an existing column and its lag versions.

    Each expression computes the difference between the current value
    and the lagged value (e.g., col - col_lag_1).

    Parameters
    ----------
    column : str
        Column name to generate delta expressions for.
    n_lags : int, optional
        Number of delta columns to generate. Default is 1.

    Returns
    -------
    list[str]
        List of SQL expressions for delta columns.

    Example
    -------
    >>> generate_delta_columns("account_balance", 2)
    [
        "account_balance - account_balance_lag_1 AS delta_1_account_balance",
        "account_balance - account_balance_lag_2 AS delta_2_account_balance"
    ]
    """
    return [
        f"{column} - {column}_lag_{i} AS delta_{i}_{column}"
        for i in range(1, n_lags + 1)
    ]


In [11]:
def generate_ratio_columns(pairs: list[list[str]]) -> list[str]:
    """
    Generate SQL expressions for ratio (amount-to-quantity) columns.

    Each pair in `pairs` should contain two related columns:
    [amount_column, quantity_column].

    The resulting expression safely divides the amount by the quantity,
    returning 0 when the denominator is 0.

    Parameters
    ----------
    pairs : list[list[str]]
        List of [amount_column, quantity_column] pairs.

    Returns
    -------
    list[str]
        List of SQL expressions for ratio columns.

    Example
    -------
    >>> generate_ratio_columns([["mcompra_total", "ccompra_total"]])
    [
        "IF(ccompra_total = 0, 0, mcompra_total / ccompra_total) AS ratio_mcompra_total_ccompra_total"
    ]
    """
    return [
        f"IF({q_col} = 0, 0, {m_col} / {q_col}) AS ratio_{m_col}_{q_col}"
        for m_col, q_col in pairs
    ]


In [12]:
def generate_linear_regression_columns(columns: list[str]) -> list[str]:
    """
    Generate SQL expressions for linear regression slope features.

    Each expression computes the slope of the regression line between
    a feature column and 'cliente_antiguedad' over a rolling 3-row window
    per client.

    Uses DuckDB's regr_slope() function with a named window 'window_3'.

    Parameters
    ----------
    columns : list[str]
        List of column names to generate regression slope features for.

    Returns
    -------
    list[str]
        List of SQL expressions for linear regression slope columns.

    Example
    -------
    >>> generate_linear_regression_columns(["mcuenta_corriente", "mpagado"])
    [
        "regr_slope(mcuenta_corriente, cliente_antiguedad) OVER window_3 AS slope_mcuenta_corriente",
        "regr_slope(mpagado, cliente_antiguedad) OVER window_3 AS slope_mpagado"
    ]
    """
    return [
        f"regr_slope({col}, cliente_antiguedad) OVER window_3 AS slope_{col}"
        for col in columns
    ]


In [13]:
def build_query(
    table_name: str,
    lag_delta_columns: list[str],
    ratio_pairs: list[list[str]] | None = None,
    linreg_columns: list[str] | None = None,
    n_lags: int = 1
) -> str:
    """
    Build a full SQL query that selects all original columns,
    plus lag, delta, ratio, and linear regression features.

    Parameters
    ----------
    table_name : str
        Name of the DuckDB table or registered relation.
    lag_delta_columns : list[str]
        Columns to generate lag and delta features for.
    ratio_pairs : list[list[str]], optional
        List of [amount_column, quantity_column] pairs for ratio computation.
    linreg_columns : list[str], optional
        Columns to generate regression slope features for.
    n_lags : int, optional
        Number of lag and delta periods to generate. Default is 1.

    Returns
    -------
    str
        Full SQL query string.
    """

    sql = "SELECT *"

    # --- Add lag and delta columns ---
    for col in lag_delta_columns:
        lag_exprs = generate_lag_columns(col, n_lags)
        delta_exprs = generate_delta_columns(col, n_lags)
        for expr in lag_exprs + delta_exprs:
            sql += f", {expr}"

    # --- Add ratio columns (if provided) ---
    if ratio_pairs:
        ratio_exprs = generate_ratio_columns(ratio_pairs)
        for expr in ratio_exprs:
            sql += f", {expr}"

# --- Linear Regressions ---
    linreg_exprs = []
    if linreg_columns:
        linreg_exprs = generate_linear_regression_columns(linreg_columns)
        for expr in linreg_exprs:
            sql += f", {expr}"

    sql += f" FROM {table_name}"

    if linreg_exprs:
        sql += """
        WINDOW window_3 AS (
            PARTITION BY numero_de_cliente
            ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
        )
        """

    return sql

# sql = build_query(table_name, lagDeltaColumns, ratioColumns, lagDeltaColumns, n_lags=2)
fe_table_name =  f"{table_name}_fe"
# store_table_sql = f"""
# CREATE OR REPLACE TABLE {table_name} AS
# {sql}
# """
# con.execute(store_table_sql)
#df_result = con.execute(sql)

sql = build_query(table_name, lagDeltaColumns, ratioColumns, lagDeltaColumns, n_lags=2)

con.execute("INSTALL sqlite;")
con.execute("LOAD sqlite;")
sqlite_path = f"{dataset_path}{fe_table_name}.db"

con.execute(f"""
ATTACH DATABASE '{sqlite_path}' AS sqlite_db (TYPE SQLITE);
CREATE OR REPLACE TABLE sqlite_db.{fe_table_name} AS
{sql};
DETACH DATABASE sqlite_db;
""")


In [14]:
%%sql
INSTALL sqlite;
LOAD sqlite;

,Success


In [15]:
%%sql
CREATE OR REPLACE TABLE {{fe_table_name}} AS
SELECT *
FROM sqlite_scan('{{dataset_path}}{{table_name}}_fe.db', '{{table_name}}_fe');

,Success


In [16]:
%%sql
select
    *
    from {{fe_table_name}}

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,slope_cliente_vip,slope_internet,slope_tcallcenter,slope_tcuentas,slope_thomebanking,slope_tmobile_app,slope_Master_delinquency,slope_Visa_delinquency,slope_Master_status,slope_Visa_status
0,249320580,202101,1,0,0,63,279,508.80,17140.27,896.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249320580,202102,1,0,0,63,280,2494.79,18472.46,2015.94,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,249320580,202103,1,0,0,63,281,3207.66,19030.88,2519.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,249320580,202104,1,0,0,64,282,3122.15,19147.05,2472.63,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,249320580,202105,1,0,0,64,283,3455.86,20443.48,2520.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1594978637,202106,1,0,0,40,2,-235.10,-228.04,-423.19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
978435,1597251555,202106,0,0,0,28,1,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978436,1597472675,202106,1,0,0,25,1,53.51,53.51,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978437,1598127775,202106,0,0,0,53,1,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
